In [1]:
import pandas as pd
from epiweeks import Week
from datetime import datetime, date
from bulletin.data.notifica import Notifica
from bulletin import __file__ as __root__
from bulletin.metabase.request import download_metabase

In [2]:
notifica = Notifica()

In [3]:
notifica.download_todas_notificacoes()

Requesting classificacao_final IS NULL AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt.status_notificacao = '2')
Success code 200
Saving in input\queries_exames\null.csv
Download finish, time elapsed: 0:00:00.179318
downloaded shape (16, 4)

Requesting classificacao_final = 0 AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt.status_notificacao = '2')
Success code 200
Saving in input\queries_exames\0.csv
Download finish, time elapsed: 0:00:00.054031
downloaded shape (12, 4)

Requesting classificacao_final = 1 AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt

In [4]:
notifica.read_todas_notificacoes()

In [5]:
todos_exames = notifica.get_casos()

In [6]:
todos_exames = todos_exames.loc[~todos_exames['ibge_unidade_notifica'].isnull()]
todos_exames = todos_exames.loc[~todos_exames['data_notificacao'].isnull()]
todos_exames['SE'] = todos_exames['data_notificacao'].apply(lambda x: Week.fromdate(x))

In [7]:
tabela = todos_exames.groupby(['SE','classificacao_final'])[['metodo']].count().unstack()

In [8]:
tabela = tabela.droplevel(0, axis=1)

In [9]:
for row in tabela.index.values:
    if row.year < 2020:
        tabela = tabela.drop(row)

In [10]:
# tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values]
# tabela = tabela.set_index('Semana Epidemiológica')

In [11]:
tabela[5] = tabela[5].add(tabela[0])
tabela = tabela.drop(0, axis=1)

In [12]:
tabela = tabela[[2, 3, 1, 5]]
tabela = tabela.rename(columns={2:'Casos confirmados', 3:'Casos descartados', 1:'Casos suspeitos', 5:'Não informados'})

In [13]:
tabela['Total testes'] = tabela.sum(1)
tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])

In [14]:
tabela.to_excel('output/TABELA_testes_notifica.xlsx')

In [15]:
tabela

classificacao_final,Casos confirmados,Casos descartados,Casos suspeitos,Não informados,Total testes,Positividade (%)
SE,,,,,,
202001,4.0,2.0,NaN,NaN,6.0,0.666667
202002,2.0,1.0,1.0,NaN,4.0,0.500000
202003,3.0,1.0,NaN,NaN,4.0,0.750000
202004,3.0,2.0,NaN,NaN,5.0,0.600000
202005,4.0,3.0,NaN,NaN,7.0,0.571429
...,...,...,...,...,...,...
202114,19937.0,17042.0,13238.0,NaN,50217.0,0.397017
202115,20143.0,17926.0,12546.0,NaN,50615.0,0.397965
202116,17198.0,14490.0,10286.0,NaN,41974.0,0.409730
